![image.png](attachment:image.png)

# Tarefa

Carregue a base ```NFP.ftr```. Essa base é proveniente do projeto #AMABiliDados, e contém dados da nota fiscal paulista cadastrados para doação automática para a AMA, Associação de Amigos do Autista.

A *Nota Fiscal Paulista* é um programa de incentivo do governo do estado de São Paulo, que retorna um pedacinho do ICMS ao consumidor sempre que solicita o registro do seu CPF na nota. Nesse programa, o contribuinte pode direcionar seus créditos a uma ONG, e quando faz isso, sempre que faz esse consumidor cadastrado faz uma compra, os créditos de notas emitidas pelo mesmo estabelecimento não identificadas (notas sem CPF) são "arrastados" para a ONG na forma de uma doação.

Clique no link, caso queira saber mais sobre a [AMA](https://www.ama.org.br/site/).<br>
Clique no link, caso queira saber como o cidadão pode ser um [doador da NFP](https://doacao1.ama.org.br/sitenfp).

Essa base possui dados das notas cujos contribuintes direcionaram seus créditos à AMA. Seus campos estão descritos abaixo:

|Campo|Descrição|
|:-|:-|
|CNPJ emit.| CNPJ do emitente da nota|
|Emitente| Nome fantasia do emitente da nota|
|No.| Número da nota fiscal|
|Data Emissão| Data de emissão da nota fiscal|
|Valor NF| Valor da nota fiscal|
|Data Registro| Data de registro no sistema da NFP|
|Créditos| Valor dos créditos (doação)|
|Situação do Crédito| Se o crédito já foi pago, está sendo processado etc.|
|Ano| Ano da emissão da nota|
|Semestre| Semestre da emissão da nota|
|Retorno| Valor do crédito dividido pelo valor da nota|
|flag_credito| Indicadora se a nota possui crédito positivo|
|categoria| Categorização da nota |

**OBJETIVO:** Algumas notas não dão retorno, o que pode ocorrer por diversos motivos, um deles seria a ocorrência de produtos não incentivados como cigarros e bebidas alcólicas. O nosso objetivo é tentar prever que tipo de nota tem maior ou menor propensão a fornecer créditos.

**Observação**: Esta é uma base real, havendo características de acordo. Pode haver uma variabilidade difícil de explicar, em parte por causa de uma freqüência de notas inconstante no tempo (por diversos motivos), em parte por haverem informações importantes, protegidas pela LGPD, não constantes na base, que pode interferir no crédito gerado, o que vai ocasionar uma variabilidade difícil de se explicar. *Welcome to the jungle*, A vida real é assim :)

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:


df = pd.read_pickle('/content/drive/MyDrive/base_nfp.pkl')
df.head()

,CNPJ emit.,Emitente,No.,Data Emissão,Valor NF,Data Registro,Créditos,Situação do Crédito,Ano,Semestre,Retorno,flag_credito,categoria
0,03.476.811/0741-98,DIA BRASIL SOCIEDADE LIMITADA,537,2017-06-30,24.68,2017-06-30,0.00,Liberado,2017,1,0.000000,0,Mercado
1,43.101.310/0001-05,AUTO POSTO ZANWAL LTDA EPP,2894,2017-06-30,50.00,2017-06-30,0.00,Liberado,2017,1,0.000000,0,Auto posto
2,23.750.257/0001-66,OFICINA DO TRIGO PASTIFICIO PANIFICADORA BAR E RE,699,2017-06-30,27.00,2017-06-30,0.44,Liberado,2017,1,0.016296,1,Restaurantes
3,23.750.257/0001-66,OFICINA DO TRIGO PASTIFICIO PANIFICADORA BAR E RE,698,2017-06-30,111.50,2017-06-30,1.81,Liberado,2017,1,0.016233,1,Restaurantes
4,23.750.257/0001-66,OFICINA DO TRIGO PASTIFICIO PANIFICADORA BAR E RE,694,2017-06-30,125.00,2017-06-30,2.03,Liberado,2017,1,0.016240,1,Restaurantes


## Análise da probabilidade de retorno

Considere somente os dados de janeiro de 2020 em diante para esta análise.

- Analise a proporção de notas que tiveram retorno>0 por categoria de estabelecimento. Esta análise pode ser por tabela ou gráfico.

In [46]:
#Considere somente os dados de janeiro de 2020 em diante para esta análise.

base_dados = df.loc[df['Data Emissão'] > '2020-01-01']

In [47]:
# criando coluna booleana para retorno > 0 ("retorno_maior")

base_dados.loc[base_dados['Retorno'] > 0, 'retorno_maior'] = True
base_dados.loc[base_dados['Retorno'] <= 0, 'retorno_maior'] = False



<ipython-input-47-1957bd766ec2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_dados.loc[base_dados['Retorno'] > 0, 'retorno_maior'] = True


In [48]:
# Analise a proporção de notas que tiveram retorno>0 por categoria de estabelecimento ("proporcao_retorno_maior")

tab = pd.crosstab(base_dados.categoria, base_dados.retorno_maior, margins=True, margins_name='total')
tab['proporcao_retorno_maior'] = tab[True]/tab['total']
tab

retorno_maior,False,True,total,proporcao_retorno_maior
categoria,,,,
Alimentos,1605,3439,5044,0.681800
Auto posto,403,135,538,0.250929
Construção,8520,7522,16042,0.468894
Farmácia,6601,3582,10183,0.351763
Mercado,14373,13176,27549,0.478275
Restaurantes,552,1625,2177,0.746440
Varejo,3481,701,4182,0.167623
Vestuário,1472,443,1915,0.231332
não definido,11658,6688,18346,0.364548


## Cálculo do WOE

- Calcule o WOE das categorias, sendo que o evento em questão é a nota ter retorno>0.
- Analise este WOE por tabela e por gráfico.

In [49]:
# Calcule o WOE das categorias, sendo que o evento em questão é a nota ter retorno>0.

tab['pct_nao_evento'] = tab[False]/tab.loc['total',False]
tab['pct_evento']     = tab[True]/tab.loc['total',True]
tab['RC'] = tab['pct_evento']/tab['pct_nao_evento']
tab['WOE'] = np.log(tab['pct_evento']/tab['pct_nao_evento'])

In [50]:
# Analise este WOE por tabela e por gráfico

tab

retorno_maior,False,True,total,proporcao_retorno_maior,pct_nao_evento,pct_evento,RC,WOE
categoria,,,,,,,,
Alimentos,1605,3439,5044,0.681800,0.032981,0.092171,2.794711,1.027729
Auto posto,403,135,538,0.250929,0.008281,0.003618,0.436927,-0.827990
Construção,8520,7522,16042,0.468894,0.175074,0.201603,1.151526,0.141088
Farmácia,6601,3582,10183,0.351763,0.135642,0.096004,0.707776,-0.345628
Mercado,14373,13176,27549,0.478275,0.295346,0.353140,1.195683,0.178717
Restaurantes,552,1625,2177,0.746440,0.011343,0.043553,3.839672,1.345387
Varejo,3481,701,4182,0.167623,0.071530,0.018788,0.262660,-1.336895
Vestuário,1472,443,1915,0.231332,0.030248,0.011873,0.392533,-0.935136
não definido,11658,6688,18346,0.364548,0.239556,0.179250,0.748259,-0.290006


## Cálculo do *Information Value*

Calcule o *IV* dessa variável.

In [51]:
# Calcule o IV dessa variável.

tab['iv_parcial'] = (tab.pct_evento - tab.pct_nao_evento)*tab.WOE
print(tab.iv_parcial.sum())
tab

0.24098114935091772


retorno_maior,False,True,total,proporcao_retorno_maior,pct_nao_evento,pct_evento,RC,WOE,iv_parcial
categoria,,,,,,,,,
Alimentos,1605,3439,5044,0.681800,0.032981,0.092171,2.794711,1.027729,0.060832
Auto posto,403,135,538,0.250929,0.008281,0.003618,0.436927,-0.827990,0.003861
Construção,8520,7522,16042,0.468894,0.175074,0.201603,1.151526,0.141088,0.003743
Farmácia,6601,3582,10183,0.351763,0.135642,0.096004,0.707776,-0.345628,0.013700
Mercado,14373,13176,27549,0.478275,0.295346,0.353140,1.195683,0.178717,0.010329
Restaurantes,552,1625,2177,0.746440,0.011343,0.043553,3.839672,1.345387,0.043335
Varejo,3481,701,4182,0.167623,0.071530,0.018788,0.262660,-1.336895,0.070510
Vestuário,1472,443,1915,0.231332,0.030248,0.011873,0.392533,-0.935136,0.017183
não definido,11658,6688,18346,0.364548,0.239556,0.179250,0.748259,-0.290006,0.017489


## Conclua

Não se esqueça de registrar suas conclusões em uma célula de texto. <font color='red'>**Sugestão:**</font> tente alterar a cor da fonte nesta célula para diferenciá-la das demais.


In [52]:
# atribuindo o poder preditivo de cada item da categoria conforme valor do IV

tab.loc[tab['iv_parcial'] >= 0.5, 'poder_preditivo'] = 'suspeito'
tab.loc[(tab['iv_parcial'] < 0.5) & (tab['iv_parcial'] >= 0.3), 'poder_preditivo'] = 'forte'
tab.loc[(tab['iv_parcial'] < 0.3) & (tab['iv_parcial'] >= 0.1), 'poder_preditivo'] = 'medio'
tab.loc[(tab['iv_parcial'] < 0.1) & (tab['iv_parcial'] >= 0.02), 'poder_preditivo'] = 'fraco'
tab.loc[tab['iv_parcial'] < 0.02, 'poder_preditivo'] = 'inutil'

In [53]:
tab['poder_preditivo']

categoria
Alimentos        fraco
Auto posto      inutil
Construção      inutil
Farmácia        inutil
Mercado         inutil
Restaurantes     fraco
Varejo           fraco
Vestuário       inutil
não definido    inutil
total           inutil
Name: poder_preditivo, dtype: object

<font color='red'>Verifica-se acima que o itens "Restaurantes", "Verejo" e "Alimentos" tem maiores poderes preditivo da Categoria</font>